In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, ChiSqSelector, Normalizer, StringIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VarianceThresholdSelector

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import getpass


Set seed and initialize spark session

In [3]:
SEED = 42
seed = 42
random.seed(seed)
np.random.seed(seed)

spark = SparkSession \
    .builder \
    .appName("DIC EX 2 - group 36") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("Spark session initialized")



Spark session initialized


here we reuse the pipe that we created in ex2_part2


In [4]:
USER      = getpass.getuser() 
PIPE_PATH = f"hdfs:///user/{USER}/models/feature_pipe_part2" 

data_path = "hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"       
df = spark.read.json(data_path)

feat_model = PipelineModel.load(PIPE_PATH)

Py4JJavaError: An error occurred while calling o25.json.
: java.io.IOException: Incomplete HDFS URI, no host: hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json
	at org.apache.hadoop.hdfs.DistributedFileSystem.initialize(DistributedFileSystem.java:184)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.exec

we split the data in train, validation and test split. Before that we sample the dataset down to 5% due to limited computational resources.

In [ ]:
df = df.sample(withReplacement=False, fraction=0.05, seed=42) #sample to 5%

train_data, temp = df.randomSplit([0.7, 0.3], seed=SEED)

valid_data, test_data = temp.randomSplit([0.5, 0.5], seed=SEED)

print(f"train rows: {train_data.count()}")
print(f"valid rows: {valid_data.count()}")
print(f"test rows: {test_data.count()}")

train rows: 2889
valid rows: 587


[Stage 29:=============================>                            (1 + 1) / 2]

test rows: 584


as we are going to create our own filtering in the next steps we are deleting the chisquared filtering from the pipe and taking the tf_idf as our input for the selection

In [7]:
stages_no_chi = feat_model.stages
stages_no_chi = feat_model.stages
if stages_no_chi[-1].__class__.__name__ == "ChiSqSelectorModel":
    stages_no_chi = stages_no_chi[:-1] 

idf_out = stages_no_chi[-2].getOutputCol() 
print(f"idf_out: {idf_out}")

idf_out: tf_idf


just create the base svm model 

In [ ]:
base_svm = LinearSVC(
    featuresCol="norm", labelCol="label", predictionCol="prediction",
    )

here we create two different filtering approaches

- chisqselector with 2000 features

- Variance ThresholdSelector

each of them with a specific normalizer

we also create the OneVsRest for our svm

In [9]:
chisq = ChiSqSelector(
        featuresCol=idf_out, outputCol="selected", labelCol="label", numTopFeatures=2000,
    )

chi_normalizer = Normalizer(
    inputCol="selected", outputCol="norm", p=2.0
    )


In [10]:
vts = VarianceThresholdSelector(            
      featuresCol=idf_out, 
      outputCol="selected"
    )

vts_normalizer = Normalizer(inputCol="selected", outputCol="norm")

In [11]:
ovr = OneVsRest(classifier=base_svm,
                labelCol="label", featuresCol="norm")


this creates the two different pipelines for both filtering models

In [12]:
chi_pipeline = Pipeline(stages=stages_no_chi + [chisq, chi_normalizer, ovr])
vts_pipeline = Pipeline(stages=stages_no_chi + [vts, vts_normalizer, ovr])

here we initialize the ParameterGrid with different parameters for the sv,:

- max. Iterations: 5, 10
- regularization Parameter: 0.1, 1.0, 10.0
- standardization: True, False 

and create the evaluator with the F1 metric 


In [13]:
param_grid = (
    ParamGridBuilder()
        .addGrid(base_svm.maxIter, [5,10])
        .addGrid(base_svm.regParam, [0.1, 1.0, 10.0])
        .addGrid(base_svm.standardization, [True, False])
        .build()
    )

evaluator = MulticlassClassificationEvaluator(
        labelCol="label", predictionCol="prediction", metricName="f1"
    )


here we build a function to run both pipelines and safe the best parameters aswell as the F1 score, 
we then save the results and print them out

In [14]:
def run_tvs(pipe, param_grid, name):
    tvs = TrainValidationSplit(
        estimator=pipe,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        trainRatio=0.8,
        seed=SEED
    )
    print(f"\n Fitting TVS for {name} …")
    model = tvs.fit(train_data)
    val_metrics = model.validationMetrics
    best_i     = val_metrics.index(max(val_metrics))
    best_val   = val_metrics[best_i]
    best_params= tvs.getEstimatorParamMaps()[best_i]
    test_pred  = model.bestModel.transform(test_data)
    test_f1    = evaluator.evaluate(test_pred)
    print(f" {name}  Best val F1 = {best_val:.4f}  params={best_params}")
    print(f" {name}  Test F1     = {test_f1:.4f}")
    return best_val, test_f1

In [16]:
results = {}
results['var'], results['var_test'] = run_tvs(vts_pipeline, param_grid, "VTS")
results['chi2'], results['chi2_test'] = run_tvs(chi_pipeline, param_grid, "ChiSq")



 Fitting TVS for VTS …
25/05/13 01:58:33 WARN CacheManager: Asked to cache already cached data.
25/05/13 01:58:33 WARN CacheManager: Asked to cache already cached data.


25/05/13 02:00:01 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB


[Stage 1800:>                                                       (0 + 2) / 2]

25/05/13 02:00:02 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB


25/05/13 02:01:51 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB


...

25/05/13 02:54:48 WARN DAGScheduler: Broadcasting large task binary with size 1235.0 KiB


25/05/13 02:58:29 WARN DAGScheduler: Broadcasting large task binary with size 1235.0 KiB


25/05/13 03:00:57 WARN DAGScheduler: Broadcasting large task binary with size 1233.6 KiB


[Stage 21002:===========================>                           (1 + 1) / 2]

 ChiSq  Best val F1 = 0.3612  params={Param(parent='LinearSVC_6d0c326237d8', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LinearSVC_6d0c326237d8', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='LinearSVC_6d0c326237d8', name='standardization', doc='whether to standardize the training features before fitting the model.'): True}
 ChiSq  Test F1     = 0.3814


In [ ]:
spark.stop()